In [ ]:
import pandas as pd
import numpy as np

def merge_intervals(intervals):
    if not intervals:
        return []

    # Сортировка по начальному времени
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]

    for current in intervals[1:]:
        prev_start, prev_end = merged[-1]
        curr_start, curr_end = current

        # Проверка на перекрытие или соприкосновение
        if curr_start <= prev_end:
            # Объединяем интервалы
            merged[-1] = (prev_start, max(prev_end, curr_end))
        else:
            merged.append(current)

    return merged

detections = pd.read_csv('../dataset/detections.csv')
detections_call = detections[detections['label'] == 'call']
# Не нужно использовать merge_intervals сейчас он только вредит
predicted_intervals = (list(zip(detections_call['start_s'], detections_call['end_s'])))

# 2. Парсинг true_label.txt вручную
true_intervals = []
with open('../dataset/true_label.txt', 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) < 2:
            continue
        start_str = parts[0].replace(',', '.')
        end_str = parts[1].replace(',', '.')
        start = float(start_str)
        end = float(end_str)
        true_intervals.append((start, end))

# 3 IoU Глобально
iou_global = 0.5
# Выводим для проверки
print("True intervals (call):", true_intervals)
print("Detected intervals (call):", predicted_intervals)

True intervals (call): [(16.64595, 17.64888), (24.368349, 25.349689), (42.17569, 42.778351), (43.80769, 44.47702), (41.309021, 42.06636), (51.514622, 52.214931), (61.05003, 61.72472), (75.898003, 77.025993), (84.827797, 85.286003), (109.514198, 110.776497), (116.905998, 117.618103), (128.7603, 129.434601), (131.129303, 131.864105), (134.044998, 134.736801), (158.687897, 159.206604), (167.8526, 168.423004), (198.684097, 199.358398), (203.811203, 204.407898), (252.985703, 253.841599), (49.790119, 50.428871), (157.111206, 157.822998), (158.680695, 159.958206), (160.797699, 161.746704), (172.183502, 172.779694), (186.304596, 187.077194), (187.624695, 188.518906), (189.139404, 189.723404), (222.542496, 223.023102), (254.650299, 255.349899), (279.752411, 280.257294)]
Detected intervals (call): [(16.5, 17.5), (17.0, 18.0), (24.0, 25.0), (24.5, 25.5), (41.0, 42.0), (41.5, 42.5), (42.0, 43.0), (42.5, 43.5), (43.0, 44.0), (43.5, 44.5), (44.0, 45.0), (48.5, 49.5), (49.0, 50.0), (49.5, 50.5), (51.

In [20]:
import numpy as np
from tqdm import tqdm

def calculate_iou(interval_a, interval_b):
    start_a, end_a = interval_a
    start_b, end_b = interval_b
    
    intersection = max(0, min(end_a, end_b) - max(start_a, start_b))
    union = (end_a - start_a) + (end_b - start_b) - intersection
    return intersection / union if union > 0 else 0.0

def calculate_ap(true_segments, pred_segments, iou_thresholds):
    aps = []
    
    for thresh in iou_thresholds:
        # Сопоставление предсказаний с истинными интервалами
        matched_true = set()
        tp = np.zeros(len(pred_segments))
        fp = np.zeros(len(pred_segments))
        
        for i, pred in enumerate(pred_segments):
            best_iou = 0.0
            best_idx = -1
            for j, true in enumerate(true_segments):
                if j in matched_true:
                    continue
                iou = calculate_iou(pred, true)
                if iou > best_iou:
                    best_iou = iou
                    best_idx = j
            
            if best_iou >= thresh:
                matched_true.add(best_idx)
                tp[i] = 1
            else:
                fp[i] = 1

        # Рассчитываем precision-recall кривую
        tp_cumsum = np.cumsum(tp)
        fp_cumsum = np.cumsum(fp)
        
        recalls = tp_cumsum / len(true_segments)
        precisions = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-12)
        
        # Интерполяция precision для 101 точки
        interp_precision = np.zeros(101)
        for r in range(101):
            precision_vals = precisions[recalls >= r/100]
            interp_precision[r] = max(precision_vals) if len(precision_vals) > 0 else 0
        
        # Вычисляем AP как среднее значение precision
        ap = np.mean(interp_precision)
        aps.append(ap)
    
    return aps

def evaluate_metrics(true_segments, pred_segments, iou_threshold=iou_global): #Важный коэф надо покрутить
    true_positives = 0
    matched_true_indices = set()
    
    for pred in pred_segments:
        best_iou = 0.0
        best_true_idx = -1
        
        for i, true in enumerate(true_segments):
            if i in matched_true_indices:
                continue
                
            iou = calculate_iou(pred, true)
            if iou > best_iou:
                best_iou = iou
                best_true_idx = i
                
        if best_iou >= iou_threshold:
            true_positives += 1
            matched_true_indices.add(best_true_idx)
    
    false_positives = len(pred_segments) - true_positives
    false_negatives = len(true_segments) - true_positives
    print(true_positives, false_negatives, false_positives)
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return {
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1_score,
        'AP50': calculate_ap(true_segments, pred_segments, [0.5])[0],
        'AP75': calculate_ap(true_segments, pred_segments, [0.75])[0],
        'AP50-95': np.mean(calculate_ap(true_segments, pred_segments, np.arange(0.5, 1.0, 0.05)))
    }

# Пример использования
"""
true_intervals = [
(0.592229  ,1.262479),
(7.773479  ,8.491604),
(29.272560  ,29.863020),
(33.896339  ,34.614460),
(36.090611  ,36.593288),
(37.319401  ,37.806129),
(49.592899  ,50.366879),
(51.084999  ,51.866959),
(52.304119  ,52.727020),
(64.454803  ,65.069199),
(66.313957  ,66.872513),
(67.590607  ,68.133209),
(104.301201  ,105.083199),
(111.745796  ,112.288399),
(113.694801  ,114.321503)
]
predicted_intervals = [
 (0.496, 1.328),
 (7.834666666666666, 8.802666666666667),
 (29.24266666666667, 30.466666666666665),
 (33.91466666666667, 34.909333333333336),
 (35.89066666666667, 36.792),
 (37.42133333333334, 37.824),
 (43.58133333333333, 43.59733333333333),
 (49.584, 50.42133333333334),
 (50.99466666666667, 52.925333333333334),
 (64.39466666666667, 65.02666666666667),
 (66.344, 68.11733333333333),
 (104.37333333333333, 105.34933333333333),
 (111.75733333333334, 112.25333333333333),
 (113.58666666666667, 114.30133333333333),
 (115.55733333333333, 115.576)  
]
"""
metrics = evaluate_metrics(true_intervals, predicted_intervals)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

27 3 40
Precision: 0.4030
Recall: 0.9000
F1-score: 0.5567
AP50: 0.4306
AP75: 0.0215
AP50-95: 0.1439


In [21]:
def bootstrap_confidence_intervals(true_segments, pred_segments, iou_threshold=iou_global,
                                  n_bootstrap=1000, confidence_level=0.95):
    # Модифицируем для новых метрик
    n_true = len(true_segments)
    n_pred = len(pred_segments)
    
    metrics_samples = {
        'Precision': [],
        'Recall': [],
        'F1-score': [],
        'AP50': [],
        'AP75': [],
        'AP50-95': []
    }
    
    for _ in tqdm(range(n_bootstrap), desc="Bootstrapping"):
        true_bootstrap = [true_segments[i] for i in np.random.choice(n_true, n_true, replace=True)]
        pred_bootstrap = [pred_segments[i] for i in np.random.choice(n_pred, n_pred, replace=True)]
        
        metrics = evaluate_metrics(true_bootstrap, pred_bootstrap, iou_threshold)
        for k in metrics_samples:
            metrics_samples[k].append(metrics[k])
    
    # Вычисление квантилей для всех метрик
    alpha = (1 - confidence_level) / 2
    ci = {}
    
    for metric, samples in metrics_samples.items():
        ci[metric + "_CI"] = np.percentile(samples, [alpha*100, (1-alpha)*100])
    
    return ci

bootstrap = bootstrap_confidence_intervals(true_intervals, predicted_intervals)
print(bootstrap)
for metric in bootstrap:
    print(f"{metric}: {bootstrap[metric]}")

Bootstrapping:   2%|▏         | 16/1000 [00:00<00:06, 159.33it/s]

13 17 54
17 13 50
14 16 53
14 16 53
12 18 55
16 14 51
12 18 55
12 18 55
14 16 53
14 16 53
6 24 61
12 18 55
11 19 56
11 19 56
16 14 51
16 14 51
7 23 60
14 16 53
15 15 52
11 19 56
15 15 52
11 19 56
12 18 55
20 10 47
15 15 52
15 15 52
13 17 54
14 16 53
17 13 50
14 16 53
18 12 49
16 14 51
19 11 48


Bootstrapping:   5%|▌         | 50/1000 [00:00<00:05, 163.83it/s]

17 13 50
12 18 55
14 16 53
16 14 51
16 14 51
7 23 60
13 17 54
15 15 52
11 19 56
14 16 53
11 19 56
10 20 57
9 21 58
12 18 55
14 16 53
12 18 55
14 16 53


Bootstrapping:   7%|▋         | 67/1000 [00:00<00:05, 165.16it/s]

...

14 16 53
17 13 50
11 19 56
16 14 51
11 19 56
16 14 51
18 12 49
14 16 53
12 18 55
16 14 51
9 21 58
17 13 50
15 15 52
14 16 53
16 14 51
11 19 56
11 19 56
13 17 54
10 20 57
15 15 52
16 14 51
13 17 54
17 13 50
11 19 56
17 13 50
14 16 53
13 17 54
13 17 54
15 15 52
14 16 53
16 14 51
12 18 55
15 15 52
16 14 51
12 18 55
8 22 59
13 17 54
17 13 50
13 17 54
12 18 55
16 14 51
17 13 50
13 17 54
15 15 52
12 18 55
14 16 53
12 18 55
12 18 55
13 17 54
14 16 53
18 12 49
18 12 49


Bootstrapping:  95%|█████████▌| 950/1000 [00:05<00:00, 164.32it/s]

11 19 56
13 17 54
11 19 56
14 16 53
16 14 51
16 14 51
14 16 53
12 18 55
18 12 49
13 17 54
17 13 50
17 13 50
17 13 50
12 18 55
12 18 55
17 13 50
17 13 50
14 16 53
12 18 55
13 17 54
16 14 51
18 12 49
16 14 51
14 16 53
15 15 52
12 18 55
14 16 53
12 18 55
18 12 49
13 17 54
12 18 55
13 17 54
14 16 53
10 20 57
15 15 52
10 20 57
13 17 54
11 19 56
14 16 53
10 20 57
11 19 56
14 16 53
17 13 50
11 19 56
12 18 55
15 15 52
13 17 54
10 20 57
13 17 54
15 15 52
17 13 50


Bootstrapping: 100%|██████████| 1000/1000 [00:06<00:00, 164.05it/s]


13 17 54
7 23 60
10 20 57
14 16 53
11 19 56
9 21 58
19 11 48
11 19 56
16 14 51
16 14 51
15 15 52
14 16 53
13 17 54
12 18 55
14 16 53
18 12 49
12 18 55
14 16 53
13 17 54
12 18 55
14 16 53
18 12 49
14 16 53
9 21 58
14 16 53
15 15 52
19 11 48
13 17 54
17 13 50
9 21 58
14 16 53
15 15 52
18 12 49
{'Precision_CI': array([0.13432836, 0.28358209]), 'Recall_CI': array([0.3       , 0.63333333]), 'F1-score_CI': array([0.18556701, 0.39175258]), 'AP50_CI': array([0.05707973, 0.27966135]), 'AP75_CI': array([0.        , 0.04700196]), 'AP50-95_CI': array([0.01611184, 0.08917483])}
Precision_CI: [0.13432836 0.28358209]
Recall_CI: [0.3        0.63333333]
F1-score_CI: [0.18556701 0.39175258]
AP50_CI: [0.05707973 0.27966135]
AP75_CI: [0.         0.04700196]
AP50-95_CI: [0.01611184 0.08917483]
